In [1]:
import pandas as pd

# 1) 연도별 일수(윤년 여부)
day_counts = {
    2019: 365,
    2020: 366,
    2021: 365,
    2022: 365,
    2023: 365,
    2024: 366,
}

# 2) 결과를 담을 딕셔너리
avg_precip = {}

# 3) 연도별 CSV를 순회하며 평균 구하기
for year in range(2019, 2025):
    path = f"../../Data/Rain/rain_{year}.csv"
    # 한글 컬럼이 있는 공공데이터라면 CP949(=EUC-KR)로 읽어야 깨지지 않음
    rain_df = pd.read_csv(path, encoding="cp949")

    # '합계 강수량(mm)' 컬럼의 값(연간 누적 강수량)을 추출
    # 파일 구조상 한 행만 있다고 가정(여러 행일 경우 평균 내면 안 되므로, 첫 행 사용)
    total_precip = rain_df.iloc[0]["합계 강수량(mm)"]

    # (합계 강수량) ÷ (해당 연도 일수) → 일평균 강수량
    avg_precip[year] = total_precip / day_counts[year]
    print(f"{year}년 연평균 일강수량: {avg_precip[year]:.3f} mm/day")

2019년 연평균 일강수량: 2.442 mm/day
2020년 연평균 일강수량: 4.511 mm/day
2021년 연평균 일강수량: 3.251 mm/day
2022년 연평균 일강수량: 4.864 mm/day
2023년 연평균 일강수량: 4.380 mm/day
2024년 연평균 일강수량: 3.583 mm/day


In [2]:
import pandas as pd
import numpy as np
import os
import chardet

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

print("라이브러리 임포트 완료")

라이브러리 임포트 완료


In [3]:
# 파일 인코딩 자동 감지 함수
def detect_encoding(file_path, n_lines=1000):
    try:
        with open(file_path, "rb") as f:
            raw_data = b"".join([f.readline() for _ in range(n_lines)])
        return chardet.detect(raw_data)["encoding"]
    except:  # 파일이 없거나 오류 발생 시 기본 인코딩 사용
        return "utf-8"


# 윤년 확인 함수
def is_leap_year(year):
    return year % 4 == 0 and (year % 100 != 0 or year % 400 == 0)


# 연도별 데이터 처리 함수 (강수량 데이터로 변경)
def process_year(year, base_path):
    print(f"\n{year}년 데이터 처리 시작")
    sales_path = os.path.join(base_path, f"sale_{year}.csv")
    rain_path = os.path.join(base_path, f"rain_{year}.csv")  # 강수량 파일

    # 1. 상권 데이터 (원본과 동일)
    try:
        sales_enc = detect_encoding(sales_path)
        df_sales = pd.read_csv(sales_path, encoding=sales_enc)
        df_sales["연도"] = df_sales["기준_년분기_코드"].astype(str).str[:4].astype(int)
        df_sales = df_sales[df_sales["연도"] == year]
        grouped = df_sales.groupby(
            ["행정동_코드", "행정동_코드_명", "서비스_업종_코드_명"], as_index=False
        )["당월_매출_금액"].sum()
        grouped.columns = ["행정동코드", "행정동명", "업종", "총매출"]
        grouped.insert(0, "연도", year)
        print(f"{year}년 상권 데이터 (연간 집계): {grouped.shape}")
        # print(grouped.head())
    except FileNotFoundError:
        print(
            f"경고: {year}년 상권 파일({sales_path})을 찾을 수 없습니다. 이 연도는 건너<0xEB><0x9C><0x85>니다."
        )
        return pd.DataFrame()
    except Exception as e:
        print(f"경고: {year}년 상권 데이터 처리 중 오류: {e}")
        return pd.DataFrame()

    # 2. 강수량 데이터 -> 일평균 강수량 계산
    daily_avg_precipitation = 0.0
    try:
        rain_enc = detect_encoding(rain_path)
        df_rain = pd.read_csv(rain_path, encoding=rain_enc)

        if "합계 강수량(mm)" in df_rain.columns:
            # '합계 강수량(mm)' 컬럼에서 해당 연도의 누적 강수량 값을 가져옵니다.
            # rain_YYYY.csv 파일이 해당 연도의 단일 누적 강수량 값을 가진다고 가정
            # (예: 특정 지점의 연간 총 강수량)
            # 만약 여러 행이 있다면, 적절한 집계 (예: .mean() 또는 특정 행 선택) 필요
            total_precipitation_year = (
                df_rain["합계 강수량(mm)"].dropna().iloc[0]
                if not df_rain["합계 강수량(mm)"].dropna().empty
                else 0.0
            )

            days_in_year = 366 if is_leap_year(year) else 365
            if days_in_year > 0:  # 0으로 나누는 것 방지
                daily_avg_precipitation = total_precipitation_year / days_in_year
            else:
                daily_avg_precipitation = 0.0

            grouped["일평균강수량"] = round(daily_avg_precipitation, 2)  # 새 컬럼명
            print(
                f"{year}년 연간 총 강수량: {total_precipitation_year:.2f}mm, 일평균 강수량: {daily_avg_precipitation:.2f}mm"
            )
        else:
            print(
                f"경고: {year}년 강수량 파일에 '합계 강수량(mm)' 컬럼이 없습니다. 일평균강수량을 0으로 설정합니다."
            )
            grouped["일평균강수량"] = 0.0

    except FileNotFoundError:
        print(
            f"정보: {year}년 강수량 파일({rain_path})을 찾을 수 없습니다. 일평균강수량을 0으로 설정합니다."
        )
        grouped["일평균강수량"] = 0.0
    except Exception as e:
        print(
            f"경고: {year}년 강수량 데이터 처리 중 오류: {e}. 일평균강수량을 0으로 설정합니다."
        )
        if "일평균강수량" not in grouped.columns:
            grouped["일평균강수량"] = 0.0

    return grouped


# 전체 연도 처리 함수 (수정 없음, 파일명에 '강수량' 추가 가능)
def process_all_years(years, base_path):
    all_data = []
    for year_val in years:
        try:
            df = process_year(year_val, base_path)
            if not df.empty:
                # df.to_csv(os.path.join(base_path, f"통합_강수량_{year_val}.csv"), index=False, encoding='utf-8-sig')
                all_data.append(df)
                print(f"{year_val}년 처리 완료")
        except Exception as e:
            print(f"{year_val}년 처리 실패: {e}")

    if all_data:
        df_all = pd.concat(all_data, ignore_index=True)
        # 파일명 변경
        output_filename = (
            f"통합_일평균강수량_{min(years_to_process)}_{max(years_to_process)}.csv"
        )
        df_all.to_csv(
            os.path.join(base_path, output_filename), index=False, encoding="utf-8-sig"
        )
        print(f"전체 통합 파일 저장 완료: {output_filename}")
        # files.download(os.path.join(base_path, output_filename)) # 로컬 환경에서는 주석 처리
        return df_all
    return pd.DataFrame()


# 미래 값 예측 함수 (컬럼명만 주의)
def predict_future_value(
    df, col_to_predict, target_year=2025
):  # col -> col_to_predict로 변경
    if df.empty or len(df) < 2:
        print(
            f"경고: '{col_to_predict}' 예측을 위한 데이터 부족. 마지막 값 또는 0 반환."
        )
        return df[col_to_predict].iloc[-1] if not df.empty else 0.0

    X = df[["연도"]].values.reshape(-1, 1)
    y = df[col_to_predict]
    model = LinearRegression().fit(X, y)
    pred = model.predict(np.array([[target_year]]))
    return round(pred[0], 2)


print("함수 정의 완료")

함수 정의 완료


In [39]:
# 로컬 환경에 맞게 base_path 설정
# 예: base_path = 'C:/데이터분석/상권데이터'
base_path = "../../Data/Sales"

start_year = 2019
end_year = 2024
prediction_target_year = 2025
years_to_process = range(start_year, end_year + 1)

df_all = process_all_years(years_to_process, base_path)

if not df_all.empty:
    print("\n통합 데이터프레임 정보:")
    print(df_all.info())
    print("\n통합 데이터프레임 샘플:")
    print(df_all.head())
else:
    print("데이터 처리 결과가 없습니다.")


2019년 데이터 처리 시작
2019년 상권 데이터 (연간 집계): (16970, 5)
2019년 연간 총 강수량: 891.30mm, 일평균 강수량: 2.44mm
2019년 처리 완료

2020년 데이터 처리 시작
2020년 상권 데이터 (연간 집계): (17167, 5)
2020년 연간 총 강수량: 1651.10mm, 일평균 강수량: 4.51mm
2020년 처리 완료

2021년 데이터 처리 시작
2021년 상권 데이터 (연간 집계): (18037, 5)
2021년 연간 총 강수량: 1186.50mm, 일평균 강수량: 3.25mm
2021년 처리 완료

2022년 데이터 처리 시작
2022년 상권 데이터 (연간 집계): (17920, 5)
2022년 연간 총 강수량: 1775.30mm, 일평균 강수량: 4.86mm
2022년 처리 완료

2023년 데이터 처리 시작
2023년 상권 데이터 (연간 집계): (17754, 5)
2023년 연간 총 강수량: 1598.80mm, 일평균 강수량: 4.38mm
2023년 처리 완료

2024년 데이터 처리 시작
2024년 상권 데이터 (연간 집계): (17542, 5)
2024년 연간 총 강수량: 1311.40mm, 일평균 강수량: 3.58mm
2024년 처리 완료
전체 통합 파일 저장 완료: 통합_일평균강수량_2019_2024.csv

통합 데이터프레임 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105390 entries, 0 to 105389
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   연도      105390 non-null  int64  
 1   행정동코드   105390 non-null  int64  
 2   행정동명    105390 non-null  object 
 3   업종      105

In [40]:
if not df_all.empty:
    print("\n매출증감률 및 순위 계산 시작")
    df_analysis_processed = df_all.sort_values(
        by=["행정동코드", "업종", "연도"]
    ).copy()  # df_all -> df_analysis_processed (이름 변경)
    df_analysis_processed["매출증감률"] = (
        df_analysis_processed.groupby(["행정동코드", "업종"])["총매출"].pct_change()
        * 100
    )
    df_analysis_processed["순위"] = (
        df_analysis_processed.groupby(["연도", "행정동코드"])["총매출"]
        .rank(method="min", ascending=False)
        .astype(int)
    )

    # LabelEncoder는 그대로 사용
    label_encoder_업종_loaded = LabelEncoder()  # 이름 변경 (새 객체)
    df_analysis_processed["업종코드"] = label_encoder_업종_loaded.fit_transform(
        df_analysis_processed["업종"]
    )

    label_encoder_행정동_loaded = LabelEncoder()  # 이름 변경 (새 객체)
    df_analysis_processed["행정동코드_str_encoded"] = (
        label_encoder_행정동_loaded.fit_transform(
            df_analysis_processed["행정동코드"].astype(str)
        )
    )  # 원본은 '행정동코드_str'

    df_analysis_processed["전년도_총매출"] = df_analysis_processed.groupby(
        ["행정동코드", "업종"]
    )["총매출"].shift(1)

    print("\n분석 데이터프레임 정보:")
    df_analysis_processed.info()  # df_all.info() -> df_analysis_processed.info()
    print("\n분석 데이터프레임 샘플:")
    print(df_analysis_processed.head())  # df_all.head() -> df_analysis_processed.head()

    analysis_filename = (
        f"분석결과_일평균강수량_{min(years_to_process)}_{max(years_to_process)}.csv"
    )
    df_analysis_processed.to_csv(
        os.path.join(base_path, analysis_filename), index=False, encoding="utf-8-sig"
    )
    print(f"분석 결과 저장 완료: {analysis_filename}")
    # files.download(os.path.join(base_path, analysis_filename)) # 로컬 환경에서는 주석 처리
else:
    print("분석할 데이터가 없습니다.")


매출증감률 및 순위 계산 시작

분석 데이터프레임 정보:
<class 'pandas.core.frame.DataFrame'>
Index: 105390 entries, 34137 to 105389
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   연도                 105390 non-null  int64  
 1   행정동코드              105390 non-null  int64  
 2   행정동명               105390 non-null  object 
 3   업종                 105390 non-null  object 
 4   총매출                105390 non-null  int64  
 5   일평균강수량             105390 non-null  float64
 6   매출증감률              86055 non-null   float64
 7   순위                 105390 non-null  int64  
 8   업종코드               105390 non-null  int64  
 9   행정동코드_str_encoded  105390 non-null  int64  
 10  전년도_총매출            86055 non-null   float64
dtypes: float64(3), int64(6), object(2)
memory usage: 9.6+ MB

분석 데이터프레임 샘플:
         연도     행정동코드   행정동명      업종       총매출  일평균강수량       매출증감률  순위  업종코드  \
34137  2021  11110515  청운효자동      가방  22673366    3.25         

In [41]:
if not df_all.empty:  # df_all 이 아니라 df_analysis_processed 사용
    print(f"\n{prediction_target_year}년 예측 시작")
    # 연도별 평균 '일평균강수량' 데이터프레임 생성
    df_mean_env_conditions = (
        df_analysis_processed.groupby("연도")[["일평균강수량"]].first().reset_index()
    )  # df_all -> df_analysis_processed, 컬럼명 변경

    예측일평균강수량 = predict_future_value(
        df_mean_env_conditions, "일평균강수량", target_year=prediction_target_year
    )  # 컬럼명, 변수명 변경
    # 원본의 예측미세먼지는 없으므로 해당 라인 삭제

    print(f"{prediction_target_year}년 예측 일평균 강수량: {예측일평균강수량}mm/day")
    # print(f"2025년 예측 평균 미세먼지: {예측미세먼지}㎍/㎥") # 이 라인은 삭제

    df_recent_year_data = df_analysis_processed[
        df_analysis_processed["연도"] == end_year
    ].copy()  # df_all -> df_analysis_processed

    if not df_recent_year_data.empty:
        df_predict_template_2025 = (
            df_recent_year_data.copy()
        )  # df_2024 -> df_predict_template_2025
        df_predict_template_2025["연도"] = prediction_target_year
        # df_predict_template_2025['예측_총매출'] = np.nan # 이 컬럼은 모델 예측 후 생성
        df_predict_template_2025["일평균강수량"] = (
            예측일평균강수량  # 평균기온 -> 일평균강수량
        )
        # df_predict_template_2025['평균미세먼지'] = 예측미세먼지 # 이 라인은 삭제

        # 2025년의 '전년도_총매출'은 2024년(end_year)의 '총매출'
        df_predict_template_2025["전년도_총매출"] = df_predict_template_2025["총매출"]
        df_predict_template_2025["총매출"] = np.nan  # 예측 전이므로 NaN

        print(f"\n{prediction_target_year}년 예측용 데이터프레임 샘플:")
        print(df_predict_template_2025.head())
    else:
        print(f"{end_year}년 데이터가 없어 예측용 템플릿을 만들 수 없습니다.")
        df_predict_template_2025 = pd.DataFrame()  # 빈 데이터프레임으로 초기화
else:
    print("데이터가 없어 예측 준비를 할 수 없습니다.")
    df_predict_template_2025 = pd.DataFrame()  # 빈 데이터프레임으로 초기화


2025년 예측 시작
2025년 예측 일평균 강수량: 4.53mm/day

2025년 예측용 데이터프레임 샘플:
         연도     행정동코드   행정동명      업종  총매출  일평균강수량      매출증감률  순위  업종코드  \
87848  2025  11110515  청운효자동  가전제품수리  NaN    4.53 -58.195754  31     4   
87849  2025  11110515  청운효자동      문구  NaN    4.53 -22.381544  32    10   
87850  2025  11110515  청운효자동    미곡판매  NaN    4.53   1.149426  30    11   
87851  2025  11110515  청운효자동     미용실  NaN    4.53   4.274520  19    12   
87852  2025  11110515  청운효자동    반찬가게  NaN    4.53   0.000000  14    13   

       행정동코드_str_encoded     전년도_총매출  
87848                  0    33597594  
87849                  0    12265218  
87850                  0    43687643  
87851                  0  1084218623  
87852                  0  1704000000  


In [42]:
if not df_analysis_processed.empty and not df_predict_template_2025.empty:
    # 학습 데이터: 전년도_총매출이 NaN이 아닌 행만 사용
    train_df = df_analysis_processed[
        (df_analysis_processed["전년도_총매출"].notna())
        & (df_analysis_processed["총매출"].notna())
    ].copy()  # df_total -> df_analysis_processed, 총매출 NaN도 제거
    test_df_for_2025 = (
        df_predict_template_2025.copy()
    )  # df_total -> df_predict_template_2025

    if not train_df.empty:
        model_features = [
            "연도",
            "일평균강수량",
            "전년도_총매출",
            "순위",
            "업종코드",
            "행정동코드_str_encoded",
        ]  # 수정된 피처 리스트

        X_train_data = train_df[model_features]
        y_train_data = train_df["총매출"]
        X_test_data_2025 = test_df_for_2025[model_features]

        # 모델 훈련 (원본과 동일)
        rf_model_trained = RandomForestRegressor(
            n_estimators=200, random_state=42, n_jobs=-1
        )  # 모델 변수명 변경
        rf_model_trained.fit(X_train_data, y_train_data)

        # 예측 (SettingWithCopyWarning 방지 위해 .loc 사용)
        test_df_for_2025.loc[:, "예측_총매출"] = rf_model_trained.predict(
            X_test_data_2025
        )

        result_2025_df = test_df_for_2025[
            [
                "연도",
                "행정동코드",
                "행정동명",
                "업종",
                "예측_총매출",
                "일평균강수량",
                "전년도_총매출",
            ]
        ].copy()  # 컬럼명 수정
        result_2025_df.rename(
            columns={"일평균강수량": "평균강수량"}, inplace=True
        )  # 최종 컬럼명 '평균강수량'으로

        result_2025_df["매출증감률_예측"] = (
            (result_2025_df["예측_총매출"] - result_2025_df["전년도_총매출"])
            / result_2025_df["전년도_총매출"]
        ) * 100
        result_2025_df["매출증감률_예측"].replace(
            [np.inf, -np.inf], np.nan, inplace=True
        )  # 무한대 값 처리

        result_2025_df["순위"] = (
            result_2025_df.groupby("행정동코드")["예측_총매출"]
            .rank(method="min", ascending=False)
            .astype(int)
        )

        final_ordered_cols = [
            "연도",
            "행정동코드",
            "행정동명",
            "업종",
            "예측_총매출",
            "평균강수량",
            "전년도_총매출",
            "매출증감률_예측",
            "순위",
        ]
        result_2025_df = result_2025_df.reindex(
            columns=final_ordered_cols
        )  # 컬럼 순서 보장
        result_2025_df = result_2025_df.sort_values(["행정동코드", "순위"])

        print(f"\n{prediction_target_year}년 예측 결과 샘플:")
        print(result_2025_df.head())

        # 최종 결과 저장 및 다운로드
        prediction_filename = f"예측결과_일평균강수량_{prediction_target_year}.csv"
        result_2025_df.to_csv(
            os.path.join(base_path, prediction_filename),
            index=False,
            encoding="utf-8-sig",
            float_format="%.2f",
        )
        print(f"예측 결과 저장 완료: {prediction_filename}")
        # files.download(os.path.join(base_path, prediction_filename)) # 로컬 환경에서는 주석 처리
    else:
        print("학습 데이터가 없어 모델 훈련 및 예측을 진행할 수 없습니다.")
elif df_analysis_processed.empty:  # df_all 이 아니라 df_analysis_processed
    print("분석된 데이터가 없어 모델 학습 및 예측을 진행할 수 없습니다.")
else:  # df_predict_template_2025.empty 인 경우
    print(
        f"{end_year}년 데이터가 없어 {prediction_target_year}년 예측을 진행할 수 없습니다."
    )


2025년 예측 결과 샘플:
         연도     행정동코드   행정동명     업종        예측_총매출  평균강수량      전년도_총매출  \
87875  2025  11110515  청운효자동  커피-음료  2.504228e+10   4.53  22129858124   
87854  2025  11110515  청운효자동     서적  1.694517e+10   4.53  17053005856   
87878  2025  11110515  청운효자동  한식음식점  1.635661e+10   4.53  14217736458   
87862  2025  11110515  청운효자동  양식음식점  1.142312e+10   4.53  10769543081   
87858  2025  11110515  청운효자동   슈퍼마켓  7.034293e+09   4.53   7187144479   

        매출증감률_예측  순위  
87875  13.160604   1  
87854  -0.632343   2  
87878  15.043700   3  
87862   6.068726   4  
87858  -2.126735   5  
예측 결과 저장 완료: 예측결과_일평균강수량_2025.csv


/var/folders/xm/vglpnpsn6bg2p3ynj17h47pw0000gn/T/ipykernel_17946/520722472.py:55: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  result_2025_df["매출증감률_예측"].replace(
